# Initial Toxicity predictions with RoBERTa
### Running times are a main concern for later expanding how much data we use, but for now will use this as basis for building some bias detection 

In [3]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/all_data.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test_public_expanded.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test_private_expanded.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/toxicity_individual_annotations.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/train.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/identity_individual_annotations.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test.csv


In [2]:
import pandas as pd

In [3]:
filename = '/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/all_data.csv'

df = pd.read_csv(filename)

In [4]:
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
# from sklearn.feature_extraction.text import TfidfVectorizer
import re
from tqdm import tqdm
from transformers import RobertaTokenizer, RobertaModel, RobertaForSequenceClassification
import torch
from torch.utils.data import DataLoader

In [5]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=1)  # Regression task

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# Preprocessing
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

df['comment_text'] = df['comment_text'].fillna('') # Remove NaN values for TDIDF
df['cleaned_comment'] = df['comment_text'].apply(clean_text) 

In [7]:
# Separate train/test data
traindf = df[df['split'] == 'train']
testdf =  df[df['split'] == 'test']

Xtrain = traindf['cleaned_comment'] 
ytrain = traindf['toxicity'] # target
Xtest = testdf['cleaned_comment']
ytest = testdf['toxicity']

# Downsize dataset for reasonable runtimes
Xtrain = Xtrain[:1000]
Xtest = Xtest[:200]
ytrain = ytrain[:1000]
ytest = ytest[:200]


In [8]:
# Tokenize with Roberta

# As opposed to tfidf approach, do not remove stopwords for better context using roberta
Xtrain_r = traindf['cleaned_comment'].tolist()
ytrain_r = traindf['toxicity'].tolist()
Xtest_r = testdf['cleaned_comment'].tolist()
ytest_r = testdf['toxicity'].tolist()

# Downsize dataset for reasonable runtimes
Xtrain_r = Xtrain_r[:1000]
Xtest_r = Xtest_r[:200]
ytrain_r = ytrain_r[:1000]
ytest_r = ytest_r[:200]

Xtrain_encodings = tokenizer(Xtrain_r, truncation=True, padding=True, max_length=200, return_tensors='pt') # choosing length of comment
Xtest_encodings = tokenizer(Xtest_r, truncation=True, padding=True, max_length=200, return_tensors='pt')



In [9]:
# Convert to tensors to prepare for dataloader 
ytrain_tensor = torch.tensor(ytrain_r, dtype=torch.float)
ytest_tensor = torch.tensor(ytest_r, dtype=torch.float)

In [10]:
Xtraintorch = torch.utils.data.TensorDataset(Xtrain_encodings['input_ids'], Xtrain_encodings['attention_mask'], ytrain_tensor)
Xtesttorch = torch.utils.data.TensorDataset(Xtest_encodings['input_ids'], Xtest_encodings['attention_mask'], ytest_tensor)
# Try 8 batch size to reduce running time
train_dataloader = DataLoader(Xtraintorch, batch_size=8, shuffle=True)
test_dataloader = DataLoader(Xtesttorch, batch_size=8, shuffle=False)

### With batch size 8, 1000 rows of training data, 1 epoch: 10 min running time to train w Roberta

In [11]:
optimizer = torch.optim.AdamW(roberta.parameters(), lr=1e-5)
roberta.train()

for epoch in range(1):  #Testing with 1 initially 
    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()  # Clear previous gradients
        
        # Assign input data and labels from batch
        input_ids = batch[0]
        attention_mask = batch[1]
        labels = batch[2]
        
        # Forward pass: Compute predictions and loss
        outputs = roberta(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()  # backpropagate loss
        optimizer.step() 
        
    print(f"Epoch {epoch + 1}: Loss {loss.item()}")

100%|██████████| 125/125 [09:37<00:00,  4.62s/it]

Epoch 1: Loss 0.05774139612913132


In [17]:
def evaluate_model(model, test_dataloader):
    ''' Evaluate model on test data using same framework as the training loop
    Params: model: torch.nn.Module, test_dataloader: torch.dataloader 
    Output: tuple: pred (list) of predicted toxicity scores for the test data, actual (list) of true scores
    '''
    model.eval()
    pred = []
    actual = []
    
    with torch.no_grad(): # no gradient calculation for faster running 
        for batch in tqdm(test_dataloader):
            # Get the input data and labels from the batch
            input_ids = batch[0]
            attention_mask = batch[1]
            labels = batch[2]
            
            # Forward pass: Compute predictions
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            logits = outputs.logits

            pred.extend(logits.cpu().numpy())
            actual.extend(labels.cpu().numpy()) 

    return pred, actual
            
        

In [21]:
pred1, actual1 = evaluate_model(roberta, test_dataloader)

# Calculate MSE of roberta model, trained with batch size 8, 200 rows of test data, 1 epoch
mse1 = mean_squared_error(actual1, pred1)

100%|██████████| 25/25 [00:31<00:00,  1.27s/it]


In [23]:
print(f'Mean Squared Error for Roberta model using 1000/20 data split: {mse1}')

Mean Squared Error for Roberta model using 1000/20 data split: 0.06018800660967827


### Low MSE and Loss on Roberta toxicity predictions. Now addressing unintended bias in predictions: